In [1]:
# for use in tutorial and development; do not include this `sys.path` change in production:
import sys ; sys.path.insert(0, "../")

# Getting Started

First, we'll import the required libraries and add the **PyTextRank** component into the `spaCy` pipeline:

In [2]:
import pytextrank
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

Let's take a look at this pipeline now...

In [3]:
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'ner',
 'attribute_ruler',
 'lemmatizer',
 'textrank']

We can examine the `spaCy` pipeline in much greater detail...

In [4]:
nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   tok2vec           doc.tensor                                          False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

{'summary': {'tok2vec': {'assigns': ['doc.tensor'],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'tagger': {'assigns': ['token.tag'],
   'requires': [],
   'scores': ['tag_acc'],
   'retokenizes': False},
  'parser': {'assigns': ['token.dep',
    'token.head',
    'token.is_sent_start',
    'doc.sents'],
   'requires': [],
   'scores': ['dep_uas',
    'dep_las',
    'dep_las_per_type',
    'sents_p',
    'sents_r',
    'sents_f'],
   'retokenizes': False},
  'ner': {'assigns': ['doc.ents', 'token.ent_iob', 'token.ent_type'],
   'requires': [],
   'scores': ['ents_f', 'ents_p', 'ents_r', 'ents_per_type'],
   'retokenizes': False},
  'attribute_ruler': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False},
  'lemmatizer': {'assigns': ['token.lemma'],
   'requires': [],
   'scores': ['lemma_acc'],
   'retokenizes': False},
  'textrank': {'assigns': [],
   'requires': [],
   'scores': [],
   'retokenizes': False}},
 'problems': {'tok2vec': [],
  't

Next, let's load some text from a document:

In [5]:
from icecream import ic
import pathlib

text = pathlib.Path("../dat/mih.txt").read_text()
text

'Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types.\n'

Then run the `spaCy` pipeline...

In [6]:
doc = nlp(text)
len(doc)

92

Now we can access the **PyTextRank** component within the `spaCy` pipeline, and use it to get more information for post-processing of the document.
For example, let's see what the elapsed time in milliseconds was for the *TextRank* processing:

In [7]:
tr = doc._.textrank
ic(tr.elapsed_time);

ic| tr.elapsed_time: 45.09329795837402


Let's examine the top-ranked phrases in the document

In [8]:
for phrase in doc._.phrases:
    ic(phrase.rank, phrase.count, phrase.text)
    ic(phrase.chunks)

ic| phrase.rank: 0.18359439311764025
    phrase.count: 1
    phrase.text: 'mixed types'
ic| phrase.chunks: [mixed types]
ic| phrase.rank: 0.1784796193107821
    phrase.count: 3
    phrase.text: 'systems'
ic| phrase.chunks: [systems, systems, systems]
ic| phrase.rank: 0.15037838042245094
    phrase.count: 1
    phrase.text: 'minimal generating sets'
ic| phrase.chunks: [minimal generating sets]
ic| phrase.rank: 0.14740065982407313
    phrase.count: 1
    phrase.text: 'nonstrict inequations'
ic| phrase.chunks: [nonstrict inequations]
ic| phrase.rank: 0.13946027725597837
    phrase.count: 1
    phrase.text: 'strict inequations'
ic| phrase.chunks: [strict inequations]
ic| phrase.rank: 0.1195023546245721
    phrase.count: 1
    phrase.text: 'linear Diophantine equations'
ic| phrase.chunks: [linear Diophantine equations]
ic| phrase.rank: 0.11450088293222845
    phrase.count: 1
    phrase.text: 'natural numbers'
ic| phrase.chunks: [natural numbers]
ic| phrase.rank: 0.10780718173686318
    phra

## Stop Words

To show use of the *stop words* feature, first we'll output a baseline...

In [9]:
text = pathlib.Path("../dat/gen.txt").read_text()
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='words', chunks=[words, words], count=2, rank=0.16291261251069128)
ic| phrase: Phrase(text='sentences', chunks=[sentences], count=1, rank=0.1306067022269487)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.10483404051843853)
ic| phrase: Phrase(text='the remaining words', chunks=[the remaining words], count=1, rank=0.09665514724405054)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.0890015856562531)
ic| phrase: Phrase(text='gensim implements', chunks=[gensim implements], count=1, rank=0.07756577483242053)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Okapi BM25 function], count=1, rank=0.07659857245638772)
ic| phrase: Phrase(text='every other sentence', chunks=[every other sentence], count=1, rank=0.07121330443014147)
ic| phrase: Phrase(text='Okapi BM25', chunks=[Okapi BM25], count=1, rank=0.06637129156118696)
ic| phrase: Phrase(text='webpages', chunks=[webpages], count=1, ran

Notice how the top-ranked phrase above is `words` ?
Let's add that phrase to our *stop words* list, to exclude it from the ranked phrases...

In [10]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] } })

doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='sentences', chunks=[sentences], count=1, rank=0.14589095939167795)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.11251869467335666)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.09552418217461901)
ic| phrase: Phrase(text='gensim implements', chunks=[gensim implements], count=1, rank=0.08324627350503458)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Okapi BM25 function], count=1, rank=0.08221256597903477)
ic| phrase: Phrase(text='every other sentence', chunks=[every other sentence], count=1, rank=0.08004789962776512)
ic| phrase: Phrase(text='Okapi BM25', chunks=[Okapi BM25], count=1, rank=0.07123533671820845)
ic| phrase: Phrase(text='webpages', chunks=[webpages], count=1, rank=0.0702329645089719)
ic| phrase: Phrase(text='TextRank', chunks=[TextRank, TextRank, TextRank, TextRank, TextRank], count=5, rank=0.06770669955429025)
ic| phrase: Phrase(text='every sentence', chunks=

For each entry, you'll need to add a key that is the *lemma* and a value that's a list of its *part-of-speech* tags.

## Scrubber

Observe how different variations of "sentence" like "every sentence", "every other sentence" occur in phrase list. You can omit such variations by passing scrubber function in config.

In [11]:
nlp = spacy.load("en_core_web_sm")

@spacy.registry.misc("prefix_scrubber")
def prefix_scrubber():
	def scrubber_func(text: str) -> str:
		stop_prefixes = ["a", "the", "their", "every", "other"]
		return " ".join([w for w in text.split() if w not in stop_prefixes])
	return scrubber_func

nlp.add_pipe("textrank", config={ "stopwords": { "word": ["NOUN"] }, "scrubber": {"@misc": "prefix_scrubber"}})

doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='sentences', chunks=[sentences, the sentences], count=2, rank=0.14589095939167795)
ic| phrase: Phrase(text='Mihalcea et al', chunks=[Mihalcea et al], count=1, rank=0.11251869467335666)
ic| phrase: Phrase(text='text summarization', chunks=[text summarization], count=1, rank=0.09552418217461901)
ic| phrase: Phrase(text='gensim implements', chunks=[gensim implements], count=1, rank=0.08324627350503458)
ic| phrase: Phrase(text='Okapi BM25 function', chunks=[Okapi BM25 function], count=1, rank=0.08221256597903477)
ic| phrase: Phrase(text='sentence', chunks=[every sentence, every other sentence], count=2, rank=0.08004789962776512)
ic| phrase: Phrase(text='Okapi BM25', chunks=[Okapi BM25], count=1, rank=0.07123533671820845)
ic| phrase: Phrase(text='webpages', chunks=[webpages], count=1, rank=0.0702329645089719)
ic| phrase: Phrase(text='TextRank', chunks=[TextRank, TextRank, TextRank, TextRank, TextRank], count=5, rank=0.06770669955429025)
ic| phrase: Phrase(text='two s

Different variations of "sentence" are now represented as part of single entry in phrase list.

## GraphViz Export

Let's generate a GraphViz doc `lemma_graph.dot` to visualize the *lemma graph* that **PyTextRank** produced for the most recent document...

In [12]:
tr = doc._.textrank
tr.write_dot(path="lemma_graph.dot")

In [13]:
!ls -lth lemma_graph.dot

-rw-rw-r-- 1 ankush ankush 18K Jun 27 16:11 lemma_graph.dot


In [14]:
!pip install graphviz

To render this graph, you must first download `GraphViz` <https://www.graphviz.org/download/>

Then you can render a `DOT` file...

In [15]:
import graphviz as gv

gv.Source.from_file("lemma_graph.dot")

Note that the image which gets rendered in a notebook is probably "squished", but other tools can renders these as interesting graphs.

## Altair visualisation

Let's generate an interactive `altair` plot to look at the lemma graph.


In [16]:
!pip install "altair"

In [17]:
tr = doc._.textrank
tr.plot_keyphrases()

alt.Chart(...)

## Extractive Summarization

Again, working with the most recent document above, we'll summarize based on its top `15` phrases, yielding its top `5` sentences...

In [18]:
for sent in tr.summary(limit_phrases=15, limit_sentences=5):
    ic(sent)

ic| sent: First, a quick description of some popular algorithms & implementations for text summarization that exist today: the summarization module in gensim implements TextRank, an unsupervised algorithm based on weighted-graphs from a paper by Mihalcea et al.
ic| sent: Gensim’s TextRank uses Okapi BM25 function to see how similar the sentences are.
ic| sent: Create a graph where vertices are sentences.
ic| sent: It is built on top of the popular PageRank algorithm that Google used for ranking webpages.
ic| sent: In original TextRank the weights of an edge between two sentences is the percentage of words appearing in both of them.


## Using PositionRank

The *PositionRank* enhanced algorithm is simple to use in the `spaCy` pipeline and it supports all of the other features described above:

In [19]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("positionrank")

Let's load an example text:

In [20]:
text = pathlib.Path("../dat/cfc.txt").read_text()
text

" Chelsea 'opted against' signing Salomon Rondón on deadline day.\n\nChelsea reportedly opted against signing Salomón Rondón on deadline day despite their long search for a new centre forward. With Olivier Giroud expected to leave, the Blues targeted Edinson Cavani, Dries Mertens and Moussa Dembele – only to end up with none of them. According to Telegraph Sport, Dalian Yifang offered Rondón to Chelsea only for them to prefer keeping Giroud at the club. Manchester United were also linked with the Venezuela international before agreeing a deal for Shanghai Shenhua striker Odion Ighalo. Manager Frank Lampard made no secret of his transfer window frustration, hinting that to secure top four football he ‘needed’ signings. Their draw against Leicester on Saturday means they have won just four of the last 13 Premier League matches."

In [21]:
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='deadline day', chunks=[deadline day, deadline day], count=2, rank=0.1671249044190727)
ic| phrase: Phrase(text='Salomon Rondón', chunks=[Salomon Rondón, Salomon Rondón], count=2, rank=0.14836718147498046)
ic| phrase: Phrase(text='Salomón Rondón', chunks=[Salomón Rondón, Salomón Rondón], count=2, rank=0.14169986334846618)
ic| phrase: Phrase(text='Chelsea', chunks=[Chelsea, Chelsea, Chelsea, Chelsea, Chelsea, Chelsea], count=6, rank=0.13419811872859874)
ic| phrase: Phrase(text='Rondón', chunks=[Rondón], count=1, rank=0.12722264594603172)
ic| phrase: Phrase(text='a new centre', chunks=[a new centre], count=1, rank=0.09181159181129885)
ic| phrase: Phrase(text='Giroud', chunks=[Giroud, Giroud], count=2, rank=0.0783201596831592)
ic| phrase: Phrase(text='Olivier Giroud', chunks=[Olivier Giroud, Olivier Giroud], count=2, rank=0.07805316118093475)
ic| phrase: Phrase(text='none', chunks=[none], count=1, rank=0.07503538984105931)
ic| phrase: Phrase(text='their long search'

The top-ranked phrases from *PositionRank* are closely related to the "lead" items: `Chelsea`, `deadline day`, `Salomon Rondón`

Now let's re-run this pipeline with *TextRank* to compare results:

In [22]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")
doc = nlp(text)

for phrase in doc._.phrases[:10]:
    ic(phrase)

ic| phrase: Phrase(text='Shanghai Shenhua striker Odion Ighalo', chunks=[Shanghai Shenhua striker Odion Ighalo], count=1, rank=0.11863090071749424)
ic| phrase: Phrase(text='none', chunks=[none], count=1, rank=0.09802416183300769)
ic| phrase: Phrase(text='Moussa Dembele', chunks=[Moussa Dembele, Moussa Dembele], count=2, rank=0.09341044332809736)
ic| phrase: Phrase(text='deadline day', chunks=[deadline day, deadline day], count=2, rank=0.09046182507994752)
ic| phrase: Phrase(text='Dries Mertens', chunks=[Dries Mertens, Dries Mertens], count=2, rank=0.08919649435994934)
ic| phrase: Phrase(text='Edinson Cavani', chunks=[Edinson Cavani, Edinson Cavani], count=2, rank=0.08418633972470349)
ic| phrase: Phrase(text='Shanghai Shenhua', chunks=[Shanghai Shenhua], count=1, rank=0.08254442709505862)
ic| phrase: Phrase(text='Salomon Rondón', chunks=[Salomon Rondón, Salomon Rondón], count=2, rank=0.08228367707127111)
ic| phrase: Phrase(text='Salomón Rondón', chunks=[Salomón Rondón, Salomón Rondón], 

The baseline algorithm is picking up named entities, although not emphasizing the order in which these entities were introduced in the text.